In [1]:
import pickle
import numpy as np
import pandas as pd
import duration
import simple_return as sr
import statistics as sc
import stdev
from scipy.stats import kurtosis, skew
import matplotlib.pyplot as plt
import pickle

In [2]:
dt=5./(250*360)
sdt=np.sqrt(dt)
with open('/home/thegodfather/Desktop/IISER/Codes&Data2/returns.pkl','rb') as f:
    ret_five = pickle.load(f)
with open('/home/thegodfather/Desktop/IISER/Codes&Data2/nn.pkl','rb') as f:
    N = pickle.load(f)
with open('/home/thegodfather/Desktop/IISER/Codes&Data2/lz.pkl','rb') as f:
    lz = pickle.load(f)    

In [3]:
def vola(ret_m):
    ret_m=np.array(ret_m)
    L=len(ret_m)
    n=20
    new_ret_m=[np.array(ret_m[i:i+n]) for i in range(L-n)]
    Ln=len(new_ret_m)
    new_std_m=np.array([stdev.sd(new_ret_m[i]) for i in range(Ln)])
    volatility_m= new_std_m/sdt
    return(volatility_m)

In [4]:
def drift(ret_m):
    ret_m=np.array(ret_m)
    L=len(ret_m)
    n=20
    new_ret_m=[np.array(ret_m[i:i+n]) for i in range(L-n)]
    Ln=len(new_ret_m)
    new_mean_m=np.array([np.mean(new_ret_m[i]) for i in range(Ln)])
    drift_m= new_mean_m/dt
    return(drift_m)

In [5]:
# dur_m=[durationNew.durationNew(np.array(vola(*ret_five[j]))) for j in range(lz)]
dur_m=[duration.duration(np.array(vola(ret_five[j]))) for j in range(lz)]

In [6]:
vol=[np.array(vola(ret_five[j])) for j in range(lz)]
si1=[]
si2=[]

In [7]:
with open('/home/thegodfather/Desktop/IISER/Codes&Data2/Fresh/Squeeze/vol.pkl', 'wb') as f:
    pickle.dump(vol, f)

In [8]:
# svol=np.sort(vol[0])
# plt.plot(range(len(vol[0])),svol)

In [9]:
#print(len(vol))

In [10]:
tvola=[np.mean(np.array(vola(ret_five[j]))) for j in range(lz)]
tdrift=[np.mean(np.array(drift(ret_five[j]))) for j in range(lz)]
# si1=[(tvola[j]/p) for j in range(lz)]
# si2=[0.0]*(lz)

for j in range(lz):
    if(tvola[j]-((1-p)*(np.percentile(vol[j],85))))>0:
        si1.append(np.percentile(vol[j],85))
    else:
        si1.append(tvola[j]/(1-p))

for j in range(lz):
    if(tvola[j]-((1-p)*(np.min(vol[j]))))>0:
        si1.append(np.min(vol[j]))
    else:
        si1.append(tvola[j]/(1-p))

for j in range(lz):
    si1.append(0.0)

si2=[((tvola[j]-((1-p)*si1[j]))/p) for j in range(lz)]

In [11]:
m_dura=[np.mean(np.array(dur_m[j],dtype=float)) for j in range(lz)]
s_dura=[stdev.sd(np.array(dur_m[j],dtype=float)) for j in range(lz)]
w_dura=[skew(np.array(dur_m[j],dtype=float)) for j in range(lz)]
k_dura=[kurtosis(np.array(dur_m[j],dtype=float),fisher=False) for j in range(lz)]
lend=[len(np.array(dur_m[j],dtype=float)) for j in range(lz)]

In [12]:
ind=['I%02d'%(i+1) for i in range(lz)]
writer=pd.ExcelWriter('/home/thegodfather/Desktop/IISER/Codes&Data2/Fresh/Squeeze/Stat.xlsx',engine='xlsxwriter')
df=pd.DataFrame({'N':N,'Mean':m_dura,'SD':s_dura,'Skew':w_dura,'Kurtosis':k_dura,'Len':lend,'tdrift':tdrift,'tvola':tvola}, index=ind)
df.to_excel(writer,sheet_name='sheet')
writer.save()